In [2]:
import geopandas as gpd
import rasterio
import os
import numpy as np
from rasterio import features
from rasterio.features import geometry_mask

# Define the paths to your shapefile and TIFF directory
shp_file = "/home/karim/WaPOR/data/algeria/segmentation/24tiles/V3/SNIC_30000_V2.shp"
tif_directory = "/home/karim/WaPOR/data/algeria/outputs/PHENOL/"
output_file = "/home/karim/WaPOR/data/algeria/outputs/cumul/segments_eta_season1_2019.tif"

# Years of the Growing Season:
ny1, ny2, ny3 = 18, 19, 20

# Load the shapefile
gdf = gpd.read_file(shp_file)
array_length = gdf.shape[0]

# Preallocate arrays
start_array = np.full(array_length, np.nan, dtype=int)
end_array = np.full(array_length, np.nan, dtype=int)
eta_all = np.full(array_length, -99., dtype=float)

# Open the model GeoTIFF to get its projection and resolution
model = "/home/karim/WaPOR/data/algeria/segmentation/24tiles/V3/SNIC_30000_V2.tif"
with rasterio.open(model) as model_ds:
    model_crs = model_ds.crs
    model_transform = model_ds.transform
    model_width = model_ds.width
    model_height = model_ds.height

# Open the output GeoTIFF file for writing
accumulated_values = np.zeros((model_height, model_width), dtype='float32')
def get_month_and_dekade(k):
    # Ensure k is within the valid range
    if k < 1 or k > 36:
        if k == 0:
            return '00', 'D0'
        else:
            raise ValueError("k must be between 1 and 36")
    # Calculate the month and dekade
    month_index = (k - 1) // 3
    dekade = (k - 1) % 3 + 1
    # Convert month_index to a month number (1-based)
    month = month_index + 1
    # Format month as a two-digit string
    month_str = f"{month:02d}"
    dd='D' + str(dekade)

    return month_str, dd

# Iterate over the shapefile features
for idx, row in gdf.iterrows():
    segment_id = row['DN']

    # Check if the TIFF files exist
    tif_file_path = os.path.join(tif_directory, f"start_final_season_2019.tif")
    tif_file_path2 = os.path.join(tif_directory, f"end_final_season_2019.tif")

    with rasterio.open(tif_file_path) as src1, rasterio.open(tif_file_path2) as src2:
        geom = row['geometry']

        # Create a mask for the segment
        mask = features.geometry_mask([geom], out_shape=src1.shape, transform=src1.transform, invert=True)

        # Read the values from the TIFF file using the mask
        values1 = src1.read(1, masked=True)
        values2 = src2.read(1, masked=True)

        segment_values1 = values1[mask]
        segment_values2 = values2[mask]

        # Check if segment_values1 has a size greater than 0 before accessing its elements
        if len(segment_values1) > 0:
            start_array[idx] = np.asarray(segment_values1[0])
        else:
            start_array[idx] = 0

        # Check if segment_values2 has a size greater than 0 before accessing its elements
        if len(segment_values2) > 0:
            end_array[idx] = np.asarray(segment_values2[0])
        else:
            end_array[idx] = 0

        # Process ETA data
        eta = np.zeros(mask.sum(), dtype=float)  # Initialize eta with the correct shape
        nmfic = []
        nmyy = []

        n1 = start_array[idx]
        n2 = end_array[idx]
        if n1 < 0. and n2 < 0.:
            n1 = 0
            n2 = 0
            start_array[idx] = 0
            end_array[idx]   = 0

        for i in range(n1, n2 + 1):
            if i < 37:
                ssy, k = str(ny1).zfill(2), i - 0
            elif 36 < i < 73:
                ssy, k = str(ny2).zfill(2), i - 36
            elif 72 < i < 109:
                ssy, k = str(ny3).zfill(2), i - 72
            else:
                k, ssy = 0, str(0).zfill(2)
            newssy = '20' + ssy
            #print('k = ',k, ' i ',i, 'n1 = ',n1,' n2 = ',n2)
            newmm, newD = get_month_and_dekade(k)
            titr=newssy + '-' + newmm + '-' + newD
            
            nmfic.append(f"{titr}")
            nmyy.append(newssy)

        if len(nmfic) > 1 and k > 0:
            for j in range(len(nmfic)):
                input_raw_file = f'/home/karim/WaPOR/data/algeria/outputs/AET_dekad/WAPOR-3.L3-AETI-D.MIT.{nmfic[j]}.tif'
                with rasterio.open(input_raw_file) as src:
                    values1 = src.read(1, masked=True)

                    # Check if the shapes are compatible
                    if values1[mask].shape == eta.shape:
                        eta += values1[mask]
                    else:
                        # Handle the case where shapes are not compatible (adjust as needed)
                        print(f"Shapes are not compatible for segment {segment_id}")

            if eta.shape[0] > 0 and not np.isnan(eta[0]):
                eta_all[idx] = float(eta[0])
                print('Total ETA For This Segment ', idx, ' = ', eta_all[idx])
            else:
                eta_all[idx] = -99.
                print('          ', 'No season for this segment ! ', idx)

    # Process accumulated values
    segment_gdf = gdf[gdf['DN'] == segment_id]
    segment_geometry = segment_gdf.unary_union
    segment_shapes = [(segment_geometry, 1)]
    segment_mask = features.geometry_mask(
        segment_shapes,
        out_shape=(model_height, model_width),
        transform=model_transform,
        invert=True
    )
    accumulated_values += segment_mask * np.nan_to_num(eta_all[idx], nan=-99.)

# Open the output GeoTIFF file for writing
with rasterio.open(output_file, 'w', driver='GTiff', height=model_height, width=model_width, count=1,
                   dtype='float32', crs=model_crs, transform=model_transform) as dst:
    dst.write(accumulated_values, 1)

print(f"Saved segments with values to {output_file}")

/home/karim/miniconda3/lib/python3.12/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


Total ETA For This Segment  307  =  328.0
Total ETA For This Segment  308  =  139.0
Total ETA For This Segment  309  =  192.0
Total ETA For This Segment  310  =  161.0
Total ETA For This Segment  311  =  102.0
Total ETA For This Segment  312  =  176.0
Total ETA For This Segment  313  =  187.0
Total ETA For This Segment  314  =  39.0
Total ETA For This Segment  315  =  66.0
Total ETA For This Segment  316  =  48.0
Total ETA For This Segment  317  =  169.0
Total ETA For This Segment  318  =  63.0
Total ETA For This Segment  319  =  165.0
Total ETA For This Segment  320  =  47.0
Total ETA For This Segment  321  =  67.0
Total ETA For This Segment  322  =  366.0
Total ETA For This Segment  323  =  386.0
Total ETA For This Segment  324  =  243.0
Total ETA For This Segment  325  =  383.0
Total ETA For This Segment  326  =  481.0
Total ETA For This Segment  327  =  473.0
Total ETA For This Segment  328  =  519.0
Total ETA For This Segment  329  =  459.0
Total ETA For This Segment  330  =  432.

/tmp/ipykernel_4549/2500732777.py:76: RuntimeWarning: invalid value encountered in cast
  start_array[idx] = np.asarray(segment_values1[0])
/tmp/ipykernel_4549/2500732777.py:82: RuntimeWarning: invalid value encountered in cast
  end_array[idx] = np.asarray(segment_values2[0])


Total ETA For This Segment  711  =  423.0
Total ETA For This Segment  712  =  168.0
Total ETA For This Segment  713  =  277.0


/tmp/ipykernel_4549/2500732777.py:76: RuntimeWarning: invalid value encountered in cast
  start_array[idx] = np.asarray(segment_values1[0])
/tmp/ipykernel_4549/2500732777.py:82: RuntimeWarning: invalid value encountered in cast
  end_array[idx] = np.asarray(segment_values2[0])
/tmp/ipykernel_4549/2500732777.py:76: RuntimeWarning: invalid value encountered in cast
  start_array[idx] = np.asarray(segment_values1[0])
/tmp/ipykernel_4549/2500732777.py:82: RuntimeWarning: invalid value encountered in cast
  end_array[idx] = np.asarray(segment_values2[0])


...

Total ETA For This Segment  6397  =  352.0
Total ETA For This Segment  6398  =  134.0
Total ETA For This Segment  6399  =  285.0
Total ETA For This Segment  6400  =  230.0
Total ETA For This Segment  6401  =  172.0
Total ETA For This Segment  6402  =  369.0
Total ETA For This Segment  6403  =  395.0
Total ETA For This Segment  6404  =  368.0
Total ETA For This Segment  6405  =  83.0
Total ETA For This Segment  6419  =  267.0
Total ETA For This Segment  6420  =  219.0
Total ETA For This Segment  6421  =  124.0
Total ETA For This Segment  6422  =  291.0
Total ETA For This Segment  6423  =  436.0
Total ETA For This Segment  6424  =  183.0
Total ETA For This Segment  6425  =  30.0
Total ETA For This Segment  6426  =  24.0
Total ETA For This Segment  6427  =  183.0
Total ETA For This Segment  6428  =  216.0
Total ETA For This Segment  6429  =  187.0
Total ETA For This Segment  6430  =  182.0
Total ETA For This Segment  6431  =  180.0
Total ETA For This Segment  6432  =  173.0
Total ETA For 

/tmp/ipykernel_4549/2500732777.py:76: RuntimeWarning: invalid value encountered in cast
  start_array[idx] = np.asarray(segment_values1[0])
/tmp/ipykernel_4549/2500732777.py:82: RuntimeWarning: invalid value encountered in cast
  end_array[idx] = np.asarray(segment_values2[0])


Total ETA For This Segment  6514  =  311.0
Total ETA For This Segment  6515  =  300.0
Total ETA For This Segment  6516  =  143.0
Total ETA For This Segment  6517  =  262.0
Total ETA For This Segment  6518  =  139.0
Total ETA For This Segment  6519  =  340.0
Total ETA For This Segment  6520  =  331.0
Total ETA For This Segment  6521  =  187.0
Total ETA For This Segment  6522  =  300.0
Total ETA For This Segment  6523  =  40.0
Total ETA For This Segment  6524  =  233.0
Total ETA For This Segment  6525  =  353.0
Total ETA For This Segment  6526  =  332.0
Total ETA For This Segment  6527  =  339.0
Total ETA For This Segment  6528  =  416.0
Total ETA For This Segment  6529  =  311.0
Total ETA For This Segment  6530  =  169.0
Total ETA For This Segment  6531  =  253.0
Total ETA For This Segment  6532  =  461.0
Total ETA For This Segment  6533  =  234.0
Total ETA For This Segment  6534  =  138.0
Total ETA For This Segment  6535  =  177.0
Total ETA For This Segment  6536  =  234.0
Total ETA Fo

/tmp/ipykernel_4549/2500732777.py:76: RuntimeWarning: invalid value encountered in cast
  start_array[idx] = np.asarray(segment_values1[0])
/tmp/ipykernel_4549/2500732777.py:82: RuntimeWarning: invalid value encountered in cast
  end_array[idx] = np.asarray(segment_values2[0])


KeyboardInterrupt: 